In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
import json
import tensorflow.keras.preprocessing.text as tfkpt
from tensorflow.keras.preprocessing.text import Tokenizer

As far as I know the GPU thing below is useless? Also couldn't figure out running this in google colab.

In [2]:
print("Number of GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Number of GPUs Available:  0


The hotels csv seemed to be the best I could find with useful infomation columns.

In [4]:
hotels_df = pd.read_csv('./hotelreviews.csv')

In [5]:
hotels_df.head(10)

,address,categories,city,country,latitude,longitude,name,postalCode,province,reviews.date,reviews.dateAdded,reviews.doRecommend,reviews.id,reviews.rating,reviews.text,reviews.title,reviews.userCity,reviews.username,reviews.userProvince
0,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2013-09-22T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,4.0,Pleasant 10 min walk along the sea front to th...,Good location away from the crouds,NaN,Russ (kent),NaN
1,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2015-04-03T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,Really lovely hotel. Stayed on the very top fl...,Great hotel with Jacuzzi bath!,NaN,A Traveler,NaN
2,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2014-05-13T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,Ett mycket bra hotell. Det som drog ner betyge...,Lugnt l��ge,NaN,Maud,NaN
3,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2013-10-27T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,We stayed here for four nights in October. The...,Good location on the Lido.,NaN,Julie,NaN
4,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2015-03-05T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,We stayed here for four nights in October. The...,������ ���������������,NaN,sungchul,NaN
5,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2015-04-05T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,We loved staying on the island of Lido! You ne...,Very nice hotel,NaN,A Traveler,NaN
6,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2014-06-10T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,4.0,Lovely view out onto the lagoon. Excellent vie...,Lovely view out onto the lagoon. Excellent view.,NaN,A Traveler,NaN
7,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2015-05-14T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,4.0,ottimo soggiorno e ottima sistemazione nei gio...,Lovely view out onto the lagoon. Excellent view.,NaN,A Traveler,NaN
8,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2015-09-14T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,3.0,Gnstiger Ausgangspunkt fr Venedig Besuche. Ruh...,G��nstige Lage,NaN,Doppeldecker,NaN
9,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2015-05-16T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,4.0,Lidoen er perfekt til et par dages ro og afsla...,Ro og hygge,NaN,A Traveler,NaN


Below is the function for scoring the hotels. There was a 1-5 rating in my csv so I took 4 and 5 to be positive, a 3 is neutral, and 2 or 1 are negative.

In [6]:
def sentiment(rating):
    if rating >= 4:
        return int(2)
    elif rating = 3:
        return int(1)
    elif rating <= 2:
        return int(0)

In [8]:
hotels_df['sentiment'] = hotels_df['reviews.rating'].apply(sentiment)

In [9]:
def floats(score):
    return int(score)

In [10]:
hotels_df['review'] = hotels_df['reviews.rating'].dropna()
hotels_df['review'].sample(10)

2800     4.0
26808    4.0
27046    8.3
24721    4.0
3073     3.0
24822    3.0
32785    5.0
11943    5.0
23909    5.0
12235    4.0
Name: review, dtype: float64

In [11]:
first = hotels_df['reviews.rating'].dropna()
second = first.apply(floats)
second

0        4
1        5
2        5
3        5
4        5
        ..
35907    5
35908    5
35909    5
35910    0
35911    0
Name: reviews.rating, Length: 35050, dtype: int64

In [12]:
hotels_df['reviews'] = second.apply(floats)

In [13]:
hotel_reviews = hotels_df[['reviews.text', 'reviews', 'sentiment']]
hotel_reviews

,reviews.text,reviews,sentiment
0,Pleasant 10 min walk along the sea front to th...,4.0,2.0
1,Really lovely hotel. Stayed on the very top fl...,5.0,2.0
2,Ett mycket bra hotell. Det som drog ner betyge...,5.0,2.0
3,We stayed here for four nights in October. The...,5.0,2.0
4,We stayed here for four nights in October. The...,5.0,2.0
...,...,...,...
35907,Best of the Best!!!! My family and I stayed in...,5.0,2.0
35908,Amazing Vacation in Beautiful Cabin We stayed ...,5.0,2.0
35909,My husband and I stayed at Tree Tops which is ...,5.0,2.0
35910,to share your opinion of this businesswith YP ...,0.0,0.0


In [14]:
hotel_reviews.to_csv('hotel_reviews.csv')
hotel_df = pd.read_csv('hotel_reviews.csv')
hotel_df

,Unnamed: 0,reviews.text,reviews,sentiment
0,0,Pleasant 10 min walk along the sea front to th...,4.0,2.0
1,1,Really lovely hotel. Stayed on the very top fl...,5.0,2.0
2,2,Ett mycket bra hotell. Det som drog ner betyge...,5.0,2.0
3,3,We stayed here for four nights in October. The...,5.0,2.0
4,4,We stayed here for four nights in October. The...,5.0,2.0
...,...,...,...,...
35907,35907,Best of the Best!!!! My family and I stayed in...,5.0,2.0
35908,35908,Amazing Vacation in Beautiful Cabin We stayed ...,5.0,2.0
35909,35909,My husband and I stayed at Tree Tops which is ...,5.0,2.0
35910,35910,to share your opinion of this businesswith YP ...,0.0,0.0


In [15]:
hotel_df = hotel_df.dropna()
hotel_df

,Unnamed: 0,reviews.text,reviews,sentiment
0,0,Pleasant 10 min walk along the sea front to th...,4.0,2.0
1,1,Really lovely hotel. Stayed on the very top fl...,5.0,2.0
2,2,Ett mycket bra hotell. Det som drog ner betyge...,5.0,2.0
3,3,We stayed here for four nights in October. The...,5.0,2.0
4,4,We stayed here for four nights in October. The...,5.0,2.0
...,...,...,...,...
35907,35907,Best of the Best!!!! My family and I stayed in...,5.0,2.0
35908,35908,Amazing Vacation in Beautiful Cabin We stayed ...,5.0,2.0
35909,35909,My husband and I stayed at Tree Tops which is ...,5.0,2.0
35910,35910,to share your opinion of this businesswith YP ...,0.0,0.0


In [16]:
hotel_df['reviews'].apply(floats)

0        4
1        5
2        5
3        5
4        5
        ..
35907    5
35908    5
35909    5
35910    0
35911    0
Name: reviews, Length: 35028, dtype: int64

In [17]:
test_text = hotel_df['reviews.text']
test_review = hotel_df['reviews']

In [18]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(test_text)

In [19]:
test_text = tokenizer.texts_to_sequences(test_text)

In [20]:
test_text = tokenizer.sequences_to_matrix(test_text)
test_text

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 1., ..., 1., 1., 1.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [21]:
model = Sequential()

In [22]:
model.add(Dense(677, activation = 'sigmoid'))
model.add(Dropout(0.1))
model.add(Dense(577, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(281, activation='elu'))
model.add(Dropout(0.4))
model.add(Dense(167, activation='tanh'))
model.add(Dropout(0.5))

In [23]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Below is where my code stops running. I'm not sure what it's doing, but keeps giving me an index error. 

In [24]:
test_review = keras.utils.to_categorical(test_review, 4)

IndexError: index 4 is out of bounds for axis 1 with size 4

In [ ]:
model.fit(test_text, test_review, batch_size=37, epochs=10, verbose=1, validation_split=0.2, shuffle=True)

I tried to follow the instructions to the best of my ability but there is some sort of disconnect here between this and the previous attempt at machine learning. Not sure where I dropped the ball, but I don't know how to correct what I have and get my model up. 
This was very much throwing bits and pieces together and hoping it would result in the completion of the lab. *insert shrug emoji here*